# A LeetCoder's Guide to Linked Lists

## 📖 Chapter 1: Introduction to Linked Lists

Welcome to the ultimate guide for mastering Linked Lists for your coding interviews! Linked lists are fundamental data structures that appear frequently in LeetCode problems. Unlike arrays, they are not stored in contiguous memory locations.

A linked list is a chain of **nodes**. Each node contains two key pieces of information:
1.  `val`: The data or value stored in the node.
2.  `next`: A pointer (or reference) to the very next node in the chain. The last node's `next` pointer points to `None` (or `null`), signifying the end of the list.

In [1]:
# This is the standard definition for a singly-linked list node.
# We will use this class definition for all problems in this guide.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    # Optional: A helper function to print the list from this node
    def __repr__(self):
        result = []
        curr = self
        while curr:
            result.append(str(curr.val))
            curr = curr.next
        return " -> ".join(result)

## 🏃‍♂️ Chapter 2: The Two-Pointer Technique

If there is one pattern to rule them all for linked lists, it's the **Two-Pointer Technique**. Nearly every medium-to-hard linked list problem uses a variation of this pattern. It involves using two different pointers that traverse the list at different speeds or starting points to gather information.

### Variation 1: Fast & Slow Pointers

This is the most common variation. You have two pointers, `slow` and `fast`.
- `slow` moves one step at a time (`slow = slow.next`).
- `fast` moves two steps at a time (`fast = fast.next.next`).

**Common Use Cases:**
1.  **Detecting a cycle:** If the `fast` pointer ever catches up to the `slow` pointer, there's a cycle. If `fast` reaches the end (`None`), there isn't.
2.  **Finding the middle of a list:** When the `fast` pointer reaches the end of the list, the `slow` pointer will be exactly at the middle.

### Variation 2: "Ahead" & "Behind" Pointers

This variation is used to find nodes that are a certain distance from each other or from the end of the list.

**Common Use Cases:**
1.  **Finding the Nth node from the end:** Move one pointer (`ahead`) `n` steps into the list first. Then, move both `ahead` and a second pointer (`behind`) one step at a time. When `ahead` reaches the end of the list, `behind` will be at the Nth node from the end.

## 👷‍♂️ Chapter 3: The Sure-Fire Way to Build a List: The Dummy Head

A huge number of errors in linked list problems come from handling edge cases, especially related to the head of the list. What if the list is empty? What if you need to insert a node *before* the current head?

The **Dummy Node** (also called a Sentinel Node) is a game-changing pattern that elegantly solves this. It's a placeholder node that you place *before* the actual head of your list. This ensures that your list always has a node to start from, even if the "real" list is empty. This simplifies your code by eliminating the need for special checks for `head == None`.

### The Dummy Head Recipe

Here is a simple, repeatable recipe for building a new linked list:

1.  **Initialize a Dummy Head:** Create a new node that will act as the placeholder.
    `dummy = ListNode(-1)`
2.  **Create a Pointer:** Create a pointer that will be used to build the list. Initialize it to the dummy head.
    `current = dummy`
3.  **Iterate and Build:** In a loop, create your new nodes and attach them to the `current` pointer.
    `current.next = new_node`
    `current = current.next`
4.  **Return the Real Head:** At the end, the real head of your newly built list is `dummy.next`.
    `return dummy.next`

In [2]:
# Boilerplate: Building a linked list from a Python list using the Dummy Head pattern.
def build_list_from_array(arr):
    """Builds a linked list from a Python list."""
    # 1. Initialize Dummy Head
    dummy = ListNode(-1)
    
    # 2. Create a builder pointer
    current = dummy
    
    # 3. Iterate and build
    for val in arr:
        current.next = ListNode(val)
        current = current.next
        
    # 4. Return the real head
    return dummy.next

# Example usage:
my_list = build_list_from_array([1, 2, 3, 4, 5])
print(f"Built List: {my_list}")

Built List: 1 -> 2 -> 3 -> 4 -> 5


## 🔄 Chapter 4: Pattern 1: Reversal & Reordering

Reversing a linked list is a classic problem and a building block for more complex ones. The key is to meticulously manage three pointers as you iterate through the list: `prev`, `curr`, and `next`.

- `prev`: Keeps track of the node *before* the current one. It starts as `None`.
- `curr`: The current node you are processing. It starts as `head`.
- `next`: A temporary pointer to store the *next* node before you break the link.

### LeetCode Case Study: Reverse Linked List (LC #206)

The iterative solution follows a simple loop:
1.  Store the next node: `next_temp = curr.next`.
2.  Reverse the current node's pointer: `curr.next = prev`.
3.  Move `prev` up to `curr`: `prev = curr`.
4.  Move `curr` up to the stored next node: `curr = next_temp`.

When the loop finishes (`curr` is `None`), the `prev` pointer will be pointing to the new head of the reversed list.

In [3]:
from typing import Optional

class Solution_LC206:
    def reverseList_iterative(self, head: Optional[ListNode]) -> Optional[ListNode]:
        """Iterative solution for reversing a linked list."""
        prev = None
        curr = head
        
        while curr:
            # 1. Store the next node before we break the link
            next_temp = curr.next
            
            # 2. Reverse the link
            curr.next = prev
            
            # 3. & 4. Move pointers one step forward
            prev = curr
            curr = next_temp
            
        # The new head is the last node we processed, which is `prev`
        return prev

    def reverseList_recursive(self, head: Optional[ListNode]) -> Optional[ListNode]:
        """Recursive solution for reversing a linked list."""
        # Base case: if the list is empty or has one node, it's already reversed.
        if not head or not head.next:
            return head
        
        # Recursively reverse the rest of the list.
        # `reversed_head` will be the new head of the reversed sub-list.
        reversed_head = self.reverseList_recursive(head.next)
        
        # `head.next` is now the last node of the reversed sub-list.
        # We want its `next` pointer to point back to `head`.
        head.next.next = head
        
        # `head` is now the last node of the newly extended list.
        # Break its original link to avoid cycles.
        head.next = None
        
        return reversed_head

# Example usage:
solver = Solution_LC206()
my_list = build_list_from_array([1, 2, 3, 4, 5])
reversed_list = solver.reverseList_iterative(my_list)
print(f"Reversed List: {reversed_list}")

Reversed List: 5 -> 4 -> 3 -> 2 -> 1


## 🤝 Chapter 5: Pattern 2: Merging & Sorting

Merging problems involve weaving two or more linked lists together into one. The core idea is to maintain a pointer for each list and iteratively compare their values to decide which node to add next to the result list.

This is a perfect use case for the **Dummy Head** pattern. We create a dummy head to simplify building the new, merged list. We compare nodes from `list1` and `list2`, append the smaller one to our merged list, and advance the pointer of the list from which we took the node.

### LeetCode Case Study: Merge Two Sorted Lists (LC #21)

In [4]:
class Solution_LC21:
    def mergeTwoLists(self, list1: Optional[ListNode], list2: Optional[ListNode]) -> Optional[ListNode]:
        """Merges two sorted linked lists."""
        # Use the Dummy Head pattern to build the result list.
        dummy = ListNode(-1)
        current = dummy
        
        # Pointers for list1 and list2
        p1, p2 = list1, list2
        
        # Iterate while both lists have nodes
        while p1 and p2:
            if p1.val <= p2.val:
                current.next = p1
                p1 = p1.next
            else:
                current.next = p2
                p2 = p2.next
            current = current.next
            
        # At the end, one of the lists might have remaining nodes.
        # Append the rest of the non-empty list.
        if p1:
            current.next = p1
        elif p2:
            current.next = p2
            
        # The merged list starts after the dummy node.
        return dummy.next

# Example usage:
solver = Solution_LC21()
list1 = build_list_from_array([1, 2, 4])
list2 = build_list_from_array([1, 3, 4])
merged_list = solver.mergeTwoLists(list1, list2)
print(f"Merged List: {merged_list}")

Merged List: 1 -> 1 -> 2 -> 3 -> 4 -> 4


## 🌀 Chapter 6: Pattern 3: Cycle Detection

Cycle detection is a classic problem that is almost always solved with the **Fast & Slow Pointer** technique (also known as Floyd's Tortoise and Hare algorithm). 

As described in Chapter 2, we move a `slow` pointer one step at a time and a `fast` pointer two steps at a time. If there is a cycle, the `fast` pointer will eventually lap the `slow` pointer, and they will meet. If there is no cycle, the `fast` pointer will reach the end of the list.

### LeetCode Case Study: Linked List Cycle (LC #141)

In [5]:
class Solution_LC141:
    def hasCycle(self, head: Optional[ListNode]) -> bool:
        """Detects if a linked list has a cycle."""
        if not head or not head.next:
            return False
        
        slow = head
        fast = head.next
        
        while fast and fast.next:
            if slow == fast:
                return True
            slow = slow.next
            fast = fast.next.next
            
        return False

# Example Usage:
solver = Solution_LC141()
head = ListNode(3)
node2 = ListNode(2)
node0 = ListNode(0)
node4 = ListNode(-4)
head.next = node2
node2.next = node0
node0.next = node4
node4.next = node2 # This creates the cycle
print(f"Has Cycle: {solver.hasCycle(head)}")

non_cycle_list = build_list_from_array([1, 2, 3])
print(f"Has Cycle: {solver.hasCycle(non_cycle_list)}")

Has Cycle: True
Has Cycle: False


## 🗑️ Chapter 7: Pattern 4: Deletion & Removal

The core concept of deletion is simple: to delete a node, you must find its **predecessor** and change its `next` pointer to skip over the node you want to delete. For example, to delete `node B` in the sequence `A -> B -> C`, you need a pointer to `A` and set `A.next = C`.

This can be tricky at the head of the list, since the head has no predecessor. This is another prime opportunity to use the **Dummy Head** pattern. By placing a dummy node before the head, you guarantee that every node you might want to delete has a predecessor.

### LeetCode Case Study: Remove Nth Node From End of List (LC #19)

This problem is a beautiful combination of two patterns:
1.  **"Ahead & Behind" Pointers:** To find the Nth node from the end. We'll advance an `ahead` pointer `n` steps first. The node we want to *delete* is the one after the `behind` pointer when `ahead` reaches the end.
2.  **Dummy Head:** To handle the edge case where we need to remove the first node of the list. The `behind` pointer starts at the dummy node, making it the predecessor of the head.

In [6]:
class Solution_LC19:
    def removeNthFromEnd(self, head: Optional[ListNode], n: int) -> Optional[ListNode]:
        """Removes the nth node from the end of the list."""
        # 1. Use a Dummy Head to simplify edge cases (like removing the head).
        dummy = ListNode(0, head)
        
        # 2. Use the "Ahead & Behind" pointer technique.
        # `behind` will point to the node *before* the one to be deleted.
        behind = dummy
        ahead = head
        
        # 3. Move the `ahead` pointer n steps forward.
        for _ in range(n):
            ahead = ahead.next
            
        # 4. Move both pointers together until `ahead` reaches the end.
        # Now `behind` is at the predecessor of the target node.
        while ahead:
            ahead = ahead.next
            behind = behind.next
            
        # 5. Delete the node by skipping it.
        behind.next = behind.next.next
        
        return dummy.next

# Example Usage:
solver = Solution_LC19()
my_list = build_list_from_array([1, 2, 3, 4, 5])
# Remove the 2nd node from the end (the node with value 4)
new_list = solver.removeNthFromEnd(my_list, 2)
print(f"List after removing 2nd from end: {new_list}")

my_list2 = build_list_from_array([1])
# Remove the 1st node from the end (the only node)
new_list2 = solver.removeNthFromEnd(my_list2, 1)
print(f"List after removing 1st from end: {new_list2}")

List after removing 2nd from end: 1 -> 2 -> 3 -> 5
List after removing 1st from end: None


## 🧠 Chapter 8: Synthesis Problems (LeetCode Medium)

Medium and Hard LeetCode problems rarely test just one pattern. They require you to synthesize multiple techniques to solve a more complex problem. Let's look at two classic examples.

### Problem 1: Reorder List (LC #143)

**Goal:** Reorder a list `L0 → L1 → … → Ln-1 → Ln` to be `L0 → Ln → L1 → Ln-1 → …`

This problem seems complex, but it's just a sequence of patterns we've already learned. We can break it down into a clear, high-level plan:

1.  **Find the middle** of the list using the **Fast & Slow Pointer** technique. This will be our splitting point.
2.  **Split the list** into two separate halves at the middle.
3.  **Reverse the second half** of the list using the standard **Reversal** pattern.
4.  **Merge the two halves** by interleaving nodes from the first half and the reversed second half.

In [7]:
class Solution_LC143:
    def reorderList(self, head: Optional[ListNode]) -> None:
        """Reorders the list by combining multiple patterns."""
        if not head or not head.next:
            return
        
        # 1. Find the middle of the list
        slow, fast = head, head.next
        while fast and fast.next:
            slow = slow.next
            fast = fast.next.next
        
        # 2. Split the list into two halves
        second_half = slow.next
        slow.next = None # Terminate the first half
        first_half = head
        
        # 3. Reverse the second half
        prev = None
        curr = second_half
        while curr:
            next_temp = curr.next
            curr.next = prev
            prev = curr
            curr = next_temp
        second_half_reversed = prev
        
        # 4. Merge the two halves by interleaving
        p1, p2 = first_half, second_half_reversed
        while p2: # The second half can be shorter or equal in length
            # Store next nodes
            p1_next = p1.next
            p2_next = p2.next
            
            # Interleave
            p1.next = p2
            p2.next = p1_next
            
            # Move pointers
            p1 = p1_next
            p2 = p2_next
            
# Example Usage:
solver = Solution_LC143()
my_list = build_list_from_array([1, 2, 3, 4, 5])
solver.reorderList(my_list)
print(f"Reordered List: {my_list}")

Reordered List: 1 -> 5 -> 2 -> 4 -> 3


### Problem 2: Copy List with Random Pointer (LC #138)

**Goal:** Create a deep copy of a linked list where each node has a `val`, a `next` pointer, and an additional `random` pointer that can point to any node in the list or `None`.

A simple iterative copy fails because when we create a new node, we don't know where its `random` pointer should point yet—the node it's supposed to point to may not have been created. The optimal solution uses a hash map (dictionary in Python) to keep track of the relationship between old nodes and their new copies.

The plan is a two-pass approach:
1.  **First Pass:** Iterate through the original list. For each `old_node`, create its corresponding `new_node` and store the mapping in a hash map: `map[old_node] = new_node`.
2.  **Second Pass:** Iterate through the original list again. For each `old_node`, use the map to set the pointers of its copy. The `next` pointer of `new_node` is simply `map[old_node.next]`, and the `random` pointer is `map[old_node.random]`.

In [8]:
# Definition for a Node with a random pointer.
class Node:
    def __init__(self, x: int, next: 'Node' = None, random: 'Node' = None):
        self.val = int(x)
        self.next = next
        self.random = random

class Solution_LC138:
    def copyRandomList(self, head: 'Optional[Node]') -> 'Optional[Node]':
        if not head:
            return None
        
        # Hash map to store the mapping from old nodes to new nodes.
        # {old_node: new_node}
        old_to_new_map = {}
        
        # --- First Pass: Create new nodes and populate the map ---
        curr = head
        while curr:
            old_to_new_map[curr] = Node(curr.val)
            curr = curr.next
            
        # --- Second Pass: Connect the new nodes' pointers ---
        curr = head
        while curr:
            # Get the new node corresponding to the current old node
            new_node = old_to_new_map[curr]
            
            # Set the next pointer
            if curr.next:
                new_node.next = old_to_new_map[curr.next]
            
            # Set the random pointer
            if curr.random:
                new_node.random = old_to_new_map[curr.random]
                
            curr = curr.next
            
        # The head of the new list is the new node corresponding to the old head
        return old_to_new_map[head]

## ✅ Chapter 9: Final Tips for Success

### Meticulous Pointer Management: Your Golden Rules

1.  **✏️ Draw It Out:** This cannot be overstated. Before writing a single line of code for a complex pointer manipulation problem, draw the nodes on a whiteboard or piece of paper. Trace how your pointers (`prev`, `curr`, `slow`, `fast`, etc.) move with each step. This will help you catch logical errors before they become bugs.
2.  ** paranoid about Nulls:** The most common runtime error with linked lists is the dreaded null pointer exception (in Python, `AttributeError: 'NoneType' object has no attribute 'next'`). Before you ever access `node.next`, you **must** be certain that `node` itself is not `None`. This is especially important in loops and conditions (e.g., `while fast and fast.next:`).